# 프로세스와 스레드의 차이(Process vs Thread)

**프로그램**이란 파일이 저장 장치에 저장되어 있지만 메모리에 올라가 있진 않은 정적인 상태

**프로세스**란 운영체제로부터 시스템 자원을 할당받는 작업의 단위로 메모리에 올라와 실행되고 있는 프로그램의 인스턴스를 의미한다.

<br>

> 할당받는 시스템 자원의 예

- CPU 시간
- 운영되기 위해 필요한 주소 공간
- Code, Data, Stack, Heap의 구조로 되어 있는 독립된 메모리 영역

<br>

> 프로세스의 특징

- 프로세스는 각각 독립된 메모리 영역(Code, Data, Stack, Heap의 구조)을 할당 받는다.
- 프로세스당 최소 1개의 스레드(메인 스레드)를 가지고 있다
- 각 프로세스는 별도의 주소 공간에서 실행되며, 한 프로세스는 다른 프로세스의 변수나 자료구조에 접근할 수 없다.
- 한 프로세스가 다른 프로세스의 자원에 접근하려면 프로세스 간의 통신(IPC, Inter-Process Communication)을 사용해야 한다.


<그림참조>
https://github.com/boostcamp-ai-tech-4/ai-tech-interview/blob/main/answers/img/6-operating-system/process.png


**스레드(Thread)** 란 프로세스가 할당받은 자원을 이용하는 실행의 단위로 프로세스와는 다른 더 작은 실행 단위 개념으로 스레드는 프로세스의 코드에 정의된 절차에 따라 실행되는 특정한 수행 경로이다.

>스레드의 특징

- 스레드는 프로세스 내에서 각각 Stack 영역만 따로 할당받고 Code, Data, Heap 영역은 공유한다
- 스레드는 한 프로세스 내에서 동작되는 여러 실행의 흐름으로, 프로세스 내의 주소 공간이나 자원들(힙 공간등)을 같은 프로세스 내에 스레드끼리 공유하면서 실행된다.
- 같은 프로세스 안에 있는 여러 스레드들은 같은 힙 공간을 공유하지만 프로세스는 다른 프로세스의 메모리에 접근할 수 없다.
- 각각의 스레드는 별도의 레지스터와 스택을 갖고 있지만, 힙 메모리는 서로 읽고 쓸 수 있다.
- 한 스레드가 프로세스 자원을 변경하면, 다른 이웃 스레드도 그 변경 결과를 즉시 볼 수 있다.

---

### 멀티 프로세스

>정의
- 하나의 응용프로그램을 여러 개의 프로세스로 구성하여 각 프로세스가 하나의 작업을 처리하도록 하는 것이다.

>장점
- 여러 개의 자식 프로세스 중 하나에 문제가 발생하면 그 자식 프로세스만 죽는 것 이상으로 다른 영향이 확산되지 않는다.

>단점

- Context Switching에서의 오버헤드
   - CPU에서 여러 프로세스를 돌아가면서 작업을 처리하는 과정으로 동작 중인 프로세스가 대기를 하면서 해당 프로세스의 상태를 보관하고 대기하고 있던 다음 순서의 프로세스가 동작하면서 이전에 보관했던 프로세스의 상태를 복구하는 작업
   - Context Swtiching 과정에서 캐쉬 메모리 초기화 등 무거운 작업이 진행되고 많은 시간이 소모되는 등의 오버헤드가 발생하게 된다.
   - 프로세스는 각각의 독립된 메모리 영역을 할당받았기 때문에 프로세스 사이에서 공유하는 메모리가 없어, Context Switching가 발생하면 캐쉬에 있는 모든 데이터를 모두 리셋하고 다시 캐쉬정보를 불러와야 한다.

- 프로세스 사이의 어렵고 복잡한 통신 기법(IPC)
   - 프로세스는 각각의 독립된 메모리 영역을 할당받았기 때문에 하나의 프로그램에 속하는 프로세스들 사이의 변수를 공유할 수 없다.

In [3]:
"""
Multiprocessing 모듈로 멀티 프로세스 구현하기
계산을 병렬로 처리하는데 도움을 주는 것이 multiprocessing모듈로 쓰레드 대신 프로세스를 만들어서 병렬로 동작한다.
"""


from multiprocessing import Process, Queue

def work(id,start, end,result):
    total = 0
    for i in range(start, end):
        total += i

    result.put(total)
    return

if __name__ == "__main__":

    START, END = 0, 100000000
    result = Queue()
    th1 = Process(target=work, args=(1,START,END//2,result))
    th2 = Process(target=work, args=(2,END//2,END, result))

    th1.start()
    th2.start()
    th1.join()
    th2.join()

    result.put('STOP')
    total = 0
    while True:
        tmp = result.get()
        if tmp =='STOP':
            break
        else:
            total += tmp

    print(f"Result: {total}")

"""
가장 큰 장점은 threading모듈과 구현 방식이 유사하므로 기존 쓰레드 방식을 구현한 코드를 쉽게 이식할 수 있고,
변경된 Process 함수에서 객체를 받아 사용하는 것과 result로 Queue 객체를 사용하는 것이다.

물론, 프로세스는 각자가 고유한 메모리 영역을 가지므로 쓰레드에 비하면 메모리 사용이 늘어난다는 단점이 있지만, 싱글 머신 아키텍처로부터 여러 머신을 사용하는 분산 애플리케이션으로 쉽게 전환할 수 있다.

각각의 프로세스가 자신만의 메모리 공간을 사용하기 때문에 프로세스간 데이터 교환을 위해 multiprocessing.Queue 객체를 사용하고, Pipe 객체를 지원하여 데이터 교환을 돕는다.
"""

print("Process구현")


Result: 0
Process구현


Traceback (most recent call last):
  File "<string>", line 1, in <module>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/isehyeon/miniforge3/envs/Desktop/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
  File "/Users/isehyeon/miniforge3/envs/Desktop/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
    exitcode = _main(fd, parent_sentinel)
  File "/Users/isehyeon/miniforge3/envs/Desktop/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
  File "/Users/isehyeon/miniforge3/envs/Desktop/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'work' on <module '__main__' (built-in)>
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'work' on <module '__main__' (built-in)>


### 멀티 스레드

>정의
- 하나의 응용프로그램을 여러 개의 스레드로 구성하고 각 스레드로 하여금 하나의 작업을 처리하도록 하는 것이다.
- 윈도우,리눅스 등 많은 운영체제들이 멀티 프로세싱을 지원하고 있지만 멀티 스레딩을 기본으로 하고 있다.
- 웹 서버는 대표적인 멀티 스레드 응용 프로그램이다.

>장점
- 시스템 자원 소모 감소(자원의 효율성 증대)
   - 프로세스를 생성하여 자원을 할당하는 시스템 콜이 줄어들어 자원을 효율적으로 관리할 수 있다.

- 시스템 처리량 증가(처리 비용 감소)
   - 스레드 간 데이터를 주고 받는 것이 간단해지고 시스템 자원 소모가 줄어들게 된다
   - 스레드 사이의 작업량이 작아 Context Switching이 빠르다

- 간단한 통신 방법으로 인한 프로그램 응답 시간 단축
   - 스레드는 프로세스 내의 Stack영역을 제외한 모든 메모리를 공유하기 때문에 통신의 부담이 적다.


>단점

- 주의 깊은 설계가 필요하다
- 디버깅이 까다롭다
- 단일 프로세스 시스템의 경우 효과를 기대하기 어렵다
- 다른 프로세스에서 스레드를 제어할 수 없다.(즉, 프로세스 밖에서 스레드 각각을 제어할 수 없다.)
- 멀티 스레드의 경우 자원 공유의 문제가 발생한다.(동기화 문제)
- 하나의 스레드에 문제가 발생하면 전체 프로세스가 영향을 받는다.
- 자원을 공유하기 때문에 필연적으로 동기화 문제가 발생할 수 밖에 없다.


In [ ]:
"""
파이썬에서 멀티 쓰레드를 구현하는 방법은 threading 모듈을 사용하거나 thread 모듈을 사용하는 것이며, 현재 thread 모듈은 deprecated되어 threading 모듈을 사용하는 것을 권장한다.
"""

from threading import Thread

def work(id,start, end,result):
    total = 0
    for i in range(start, end):
        total += i

    result.put(total)
    return


if __name__ == "__main__":

    START, END = 0, 100000000
    result = list()
    th1 = Thread(target=work, args=(1,START,END,result))


    th1.start()
    th1.join()


"""
쓰레드는 threading 모듈의 Thread 함수로 쓰레드 객체를 받아 사용하는 것으로 target은 쓰레드가 실행할 함수, args는 그 함수의 인자들을 의미한다.
start 함수로 쓰레드를 시작하고 join함수로 쓰레드가 끝날 때까지 기다린다.
"""
print(f"Result: sum{result}")


"""
th2를 추가하여, 쓰레드에서 실행되는 함수에 들어가는 인자를 절반씩 나누어 입력하여 따로 계산하기
이 코드를 실행하면 하나의 프로세스에서 동작하지만 cpu를 가지고 잇다면 쓰레드가 적절히 분산되어 병렬 처리를 한다.
"""


if __name__ == "__main__":

    START, END = 0, 100000000
    result = list()
    th1 = Thread(target=work, args=(1,START,END,result))
    th2 = Thread(target=work, args=(2, END//2, END, result))


    th1.start()
    th1.join()
    th2.start()
    th2.join()

### GIL(Global Interoreter Lock)

언어에서 자원을 보호하기 위해 락 정책을 사용하고 그 방법 또한 다양하므로, 파이썬에선 하나의 프로세스 안에 모든 자원의 락을 글로벌하게 관리함으로써 한 번에 하나의 쓰레드만 자원을 컨트롤하여 동작하도록 하고, result 라는 자원을 공유하는 두 개의 쓰레드를 동시에 실행히시키지만, GIL 때문에 하나의 쓰레드만 계산을 실행하여 실행 시간이 비슷할 것이다.

<br>

GIL 덕분에 자원 관리를 더 쉽게 구현할 수 있었지만, 지금처럼 멀티 코어가 당연한 시대에서는 조금 아쉬운 것이 사실이지만 쓸모없진 않다.

GIL이 적용되는 것은 CPU 동작에서이고 쓰레드가 cpu 동작을 마치고 I/O 작업을 실행하는 동안에는 다른 쓰레드가 cpu 동작을 동시에 실행할 수 있기에 cpu 동작이 많지 않고 I/O 동작이 더 많은 프로그램에서 멀티 쓰레드만으로 성능적으로 큰 효과를 얻을 수 있다.

### 멀티 프로세스 대신 멀티 스레드를 사용하는 이유?

- 멀티 프로세스 대신 멀티 스레드를 사용하는 것의 의미?
   - 쉽게 설명하면, 프로그램을 여러 개 키는 것보다 하나의 프로그램 안에서 여러 작업을 해결하는 것이다.

- 여러 프로세스(멀티 프로세스)로 할 수 있는 작업들을 하나의 프로세스에서 여러 스레드로 나눠가면서 하는 이유?

   - 자원의 효율성 증대
      - 멀티 프로세스로 실행되는 작업을 멀티 스레드로 실행할 경우, 프로세스를 생성하여 자원을 할당하는 시스템 콜이 줄어들어 자원을 효율적으로 관리할 수 있다.
      - 프로세스 간의 Context Switching시 단순히 CPU 레지스터 교체 뿐만 아니라 RAM과 CPU 사이의 캐쉬 메모리에 대한 데이터까지 초기화되므로 오버헤드가 크기 때문

      - 스레드는 프로세스 내의 메모리를 공유하기 때문에 독립적인 프로세스와 달리 스레드 간 데이터를 주고 받는 것이 간단해지고 시스템 자원 소모가 줄어들게 된다.

- 처리 비용 감소 및 응답 시간 단축
   - 또한 프로세스 간의 통신(IPC)보다 스레드 간의 통신의 비용이 적으므로 작업들 간의 통신의 부담이 줄어든다.
   - 스레드는 Stack 영역을 제외한 모든 메모리를 공유하기 때문
   - 프로세스 간의 전환 속도보다 스레드 간의 전환 속도가 빠르다.
   - Context Switching시 스레드는 Stack 영역만 처리하기 때문


---


### 정말 다른 프로세스의 정보에는 접근할 수 없을까?

지금까지 안 된다고 했지만 사실 프로세스가 다른 프로세스의 정보에 접근하는 것이 가능하다. 사실 지금 우리네가 사용하는 대부분의 컴퓨터 프로그램을 생각해보면 다른 프로그램에 있는 정보를 가져오는 경우는 심심치 않게 볼 수 있다.

프로세스 간 정보를 공유하는 방법에는 다음과 같은 방법들이 있다. 다만 이 경우에는 단순히 CPU 레지스터 교체뿐만이 아니라 RAM과 CPU 사이의 캐시 메모리까지 초기화되기 때문에 앞서 말했듯 자원 부담이 크다.

IPC(Inter-Process Communication)을 사용한다.
LPC(Local inter-Process Communication)을 사용한다.

별도로 공유 메모리를 만들어서 정보를 주고받도록 설정해주면 된다.
이 글에선 해당 방법들에 대한 자세한 설명은 생략하도록 하고, 다음 기회에 더 자세하게 이 방식에 대한 글을 따로 쓰도록 하겠다.

<br>

### 결론

프로세스와 스레드는 개념의 범위부터 다르다. 스레드는 프로세스 안에 포함되어 있기 때문이다.

운영체제가 프로세스에게 Code/Data/Stack/Heap 메모리 영역을 할당해 주고 최소 작업 단위로 삼는 반면, 스레드는 프로세스 내에서 Stack 메모리 영역을 제외한 다른 메모리 영역을 같은 프로세스 내 다른 스레드와 공유한다.

프로세스는 다른 프로세스와 정보를 공유하려면 IPC를 사용하는 등의 번거로운 과정을 거쳐야 하지만, 스레드는 기본 구조 자체가 메모리를 공유하는 구조이기 때문에 다른 스레드와 정보 공유가 쉽다. 때문에 멀티태스킹보다 멀티스레드가 자원을 아낄 수 있게 된다. 다만 스레드의 스케줄링은 운영체제가 처리하지 않기 때문에 프로그래머가 직접 동기화 문제에 대응할 수 있어야 한다.